<a href="https://colab.research.google.com/github/DENGCPU/TensorFlow-Study/blob/main/%E4%BF%9D%E5%AD%98%E5%92%8C%E6%81%A2%E5%A4%8D%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyyaml h5py  # Required to save models in HDF5 format

In [4]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.15.0


In [6]:
# 获取数据集 MNIST数据集
(train_images,train_labels),(test_images,test_labels)=tf.keras.datasets.mnist.load_data()
train_labels=train_labels[:1000]
test_labels=test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [5]:
# 定义一个序列sequential模型
def create_model():
  model = tf.keras.Sequential([
      keras.layers.Dense(512,activation='relu',input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10)
      ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
                )
  return model

# 创建一个模型
model = create_model()
# 查看模型的结构
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 创建一个只在训练期间保存权重的tf.keras.callbacks.ModelCheckpoint回调
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个callback保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 # 当 verbose=1 时，每次保存模型或权重时都会在控制台输出一条消息。
                                                 verbose=1)
# 训练模型
model.fit(train_images,train_labels,epochs=10,validation_data=(test_images,test_labels),callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 0.4187 - sparse_categorical_accuracy: 0.8740
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 21ms/step - loss: 0.4187 - sparse_categorical_accuracy: 0.8740 - val_loss: 0.5515 - val_sparse_categorical_accuracy: 0.8230
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.3079 - sparse_categorical_accuracy: 0.9219
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.2990 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4788 - val_sparse_categorical_accuracy: 0.8440
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2022 - sparse_categorical_accuracy: 0.9510
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 25ms/step - loss: 0.2022 - sparse_categorical_accuracy: 0.9510 - val_loss: 0.4212 - val_sparse_categorical_accuracy: 0.8600
Epoch 4/10
29/32 [=======

In [9]:
os.listdir(checkpoint_dir)

['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']

In [10]:
# 创建一个未经过训练的新模型
model = create_model()

# 评估模型
loss,acc= model.evaluate(test_images,test_labels,verbose=2)
print('untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 2.3624 - sparse_categorical_accuracy: 0.0740 - 198ms/epoch - 6ms/step
untrained model, accuracy:  7.40%


In [11]:
# 从checkpoint 加载权重并重新评估
model.load_weights(checkpoint_path)
# 重新评估
loss,acc= model.evaluate(test_images,test_labels,verbose=2)
print('untrained model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4057 - sparse_categorical_accuracy: 0.8700 - 95ms/epoch - 3ms/step
untrained model, accuracy: 87.00%


In [12]:
# checkpoint回调选项

# 训练一个新模型，每五个epochs保存一次唯一命名的checkpoint

# 将epoch放入文件名
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size=32

# calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)

# create  a callback that saves the model's weight every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches
)
# create a model instance
model = create_model()

# 保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 训练模型
model.fit(train_images,train_labels,epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)



Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [14]:
# 检查生成的检查点
os.listdir(checkpoint_dir)

['cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.index',
 'checkpoint',
 'cp-0000.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001']

['cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.index',
 'checkpoint',
 'cp-0000.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001']

In [15]:
# 最新的检查点
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [16]:
# 创建一个新模型
model = create_model()
# 加载最新检查点的权重
model.load_weights(latest)
# 重新评估模型
loss,acc= model.evaluate(test_images,test_labels,verbose=2)
print('Restored model,accuracy: {:.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.5009 - sparse_categorical_accuracy: 0.8740 - 340ms/epoch - 11ms/step
Restored model,accuracy: 87.40%


In [17]:
# 手动保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建一个新模型
model = create_model()

model.load_weights('./checkpoints/my_checkpoint')
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print('Restored model,accuracy: {:.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.5009 - sparse_categorical_accuracy: 0.8740 - 193ms/epoch - 6ms/step
Restored model,accuracy: 87.40%


In [18]:
# 保存整个模型 tf.keras.Model.save

model= create_model()
model.fit(train_images,train_labels,epochs=5)

# 保存整个模型以.keras 的zip格式
model.save('my_model.keras')


Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.1445 - sparse_categorical_accuracy: 0.6760
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4252 - sparse_categorical_accuracy: 0.8850
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2815 - sparse_categorical_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2009 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1516 - sparse_categorical_accuracy: 0.9670


In [19]:
# 从.keras中归档重新加载新的模型
new_model = tf.keras.models.load_model('my_model.keras')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
loss, acc  = new_model.evaluate(test_images,test_labels,verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4108 - sparse_categorical_accuracy: 0.8720 - 248ms/epoch - 8ms/step
Restored model, accuracy: 87.20%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


In [21]:
# 以Savemodel 格式保存模型
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.1296 - sparse_categorical_accuracy: 0.6880
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4146 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2752 - sparse_categorical_accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2035 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 10ms/step - loss: 0.1406 - sparse_categorical_accuracy: 0.9680


In [23]:
ls saved_model/my_model

assets/  fingerprint.pb  keras_metadata.pb  saved_model.pb  variables/


In [24]:
new_model= tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
loss, acc = new_model.evaluate(test_images,test_labels,verbose=2)
print('restored model,accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4641 - sparse_categorical_accuracy: 0.8400 - 259ms/epoch - 8ms/step
restored model,accuracy: 84.00%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [27]:
# 使用HDF5格式保存模型
model=  create_model()
model.fit(train_images,train_labels,epochs=5)
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.1304 - sparse_categorical_accuracy: 0.6800
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4169 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2905 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2023 - sparse_categorical_accuracy: 0.9550
Epoch 5/5
32/32 [==============================] - 0s 9ms/step - loss: 0.1562 - sparse_categorical_accuracy: 0.9650


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
new_model =tf.keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
loss , acc = new_model.evaluate(test_images,test_labels,verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4449 - sparse_categorical_accuracy: 0.8620 - 208ms/epoch - 6ms/step
Restored model, accuracy: 86.20%
